In [2]:
from dotenv import load_dotenv
import os
import tweepy
import time
import json
import sys

def get_client():
    # load API keys
    load_dotenv()
    CONSUMER_KEY = os.getenv('CONSUMER_KEY')
    CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
    ACCESS_TOKEN= os.getenv('ACCESS_TOKEN')
    ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
    BEARER_TOKEN=os.getenv('BEARER_TOKEN')

    client = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_TOKEN_SECRET)
    user = client.get_me()
    if not user.data:
        raise Exception('[ERROR] unable to make requests with Bearer Token, check your .env file and API keys!')
    
    print(f'[INFO] authenticated using account @{user.data.username}')

    return client

def get_streaming_client():
    """
    overload methods from inherited class to customize processing of incoming stream data
    """
    class CustomSC(tweepy.StreamingClient):

        def on_tweet(self, tweet):
            print(tweet.data)
            with open('test_data/tweets.txt', 'a') as f:
                f.write(json.dumps(tweet.data)+'\n')
    
    load_dotenv()
    BEARER_TOKEN=os.getenv('BEARER_TOKEN')
    return CustomSC(bearer_token=BEARER_TOKEN)

def check_rule(streaming_client: tweepy.StreamingClient, rule: str):
    rules_response = streaming_client.get_rules()
    rules= [tweepy.StreamRule(value=rule, tag='default')]

    if not rules_response.data:
        print('[INFO] no rules found, adding rule')
        return streaming_client.add_rules(add=rules)

    elif rules_response.data and rules_response.data[0].value != rule:
        print('[INFO] updating old rule')
        streaming_client.delete_rules(rules_response.data[0].id)
        return streaming_client.add_rules(add=rules)

    elif rules_response.data[0].value == rule:
        print('[INFO] current rule matches')
        return streaming_client.get_rules()


def main(streaming_client: tweepy.StreamingClient):

    streaming_client.filter(threaded=True)
    
    i = 0
    time_to_listen = 10
    print(f'[INFO] listening for {time_to_listen} seconds...')
    for i in range(1,time_to_listen):
        time.sleep(1)
        i += 1
    print("[INFO] done streaming")
    
    streaming_client.disconnect()

    return 0




In [11]:


streaming_client = get_streaming_client()
client=get_client()

[INFO] authenticated using account @detectatron3000


In [26]:
streaming_client.filter(tweet_fields='id,text,referenced_tweets')

{'edit_history_tweet_ids': ['1598859249496444930'], 'id': '1598859249496444930', 'referenced_tweets': [{'type': 'replied_to', 'id': '1598858635698114562'}], 'text': '@sneakygollum @KomodoLord @KantoChris @TeacherTempest @D_Boi408 @JennTansley @TeejMills @shunyLucario @Rachel1991Burke @wingedaero @firemasterMT @SteemyBonBon Oh trust me I know. I caught all of mine at full health with luxury balls. I was to scared I might kill it. Even though I save I’m always hesitant to trust just closing the game and trying again. So I try not to take risks lol.'}
{'edit_history_tweet_ids': ['1598859249857093634'], 'id': '1598859249857093634', 'referenced_tweets': [{'type': 'replied_to', 'id': '1598852152327213056'}], 'text': '@amolinguas @BrassVon @BumpstockBarbie @travisdon1981 And also, the "Publican v Pharisee" parable'}
{'edit_history_tweet_ids': ['1598859252743176195'], 'id': '1598859252743176195', 'referenced_tweets': [{'type': 'replied_to', 'id': '1598776016096448514'}], 'text': '@KamalaHarris

KeyboardInterrupt: 

In [10]:
t = '{"edit_history_tweet_ids": ["1598851902548025346"], "id": "1598851902548025346", "referenced_tweets": [{"type": "retweeted", "id": "1598831435288563712"}], "text": "RT @mtaibbi: 18. Twitter took extraordinary steps to suppress the story, removing links and posting warnings that it may be \u201cunsafe.\u201d They\u2026"}'
import json 
d = json.loads(t)
d

{'edit_history_tweet_ids': ['1598851902548025346'],
 'id': '1598851902548025346',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1598831435288563712'}],
 'text': 'RT @mtaibbi: 18. Twitter took extraordinary steps to suppress the story, removing links and posting warnings that it may be “unsafe.” They…'}